# Direct Preference Optimization (DPO) at Scale with QLoRA

This guide provides a step-by-step workflow for preference fine-tuning the `Qwen/Qwen2.5-7B-Instruct` model on a multi-GPU Anyscale cluster. We will use LLaMA-Factory as the training framework and `QLoRA` to reduce memory requirements and enable efficient multi-GPU training.

**What is Direct Preference Optimization (DPO)?** DPO aligns a model with human preferences using pairs of “chosen” and “rejected” responses. Rather than training a separate reward model, DPO directly optimizes the policy to increase the likelihood of preferred outputs and decrease the likelihood of rejected ones.

## Step 1: Set Up Your Environment
### Dependencies
First, we need to ensure our environment has the right libraries. We'll start with a pre-built container image and install LLaMA-Factory and DeepSpeed on top of it.

Recommended Container Image:
```bash
anyscale/ray-llm:2.48.0-py311-cu128
```

Execute the following commands to install the required packages and optional tools for experiment tracking and faster downloads.

In [1]:
%%bash
# Install the specific version of LLaMA-Factory
pip install -q llamafactory@git+https://github.com/hiyouga/LLaMA-Factory.git@v0.9.3

# (Optional) For visualizing training metrics and logs
pip install -q tensorboard==2.20.0

# (Optional) For lightweight 8-bit and 4-bit optimizers and inference
pip install -q bitsandbytes==0.47.0

# (Optional) For AWQ quantization support
pip install -q autoawq==0.2.9

# (Optional) For accelerated model downloads from Hugging Face
pip install -q hf_transfer==0.1.9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.10.0 which is incompatible.


Successfully registered `llamafactory` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_j3li62ul9bwvaathjuulbzf7wc?workspace-tab=dependencies
Successfully registered `tensorboard` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_j3li62ul9bwvaathjuulbzf7wc?workspace-tab=dependencies
Successfully registered `bitsandbytes` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2qA8i5BjxUevf5i7/prj_cz951f43jjdybtzkx1s5sjgz99/workspaces/expwrk_j3li62ul9bwvaathjuulbzf7wc?workspace-tab=dependencies
Successfully registered `autoawq` package to be installed on all cluster nodes.
View and update dependencies here: https://console.anyscale.com/cld_kvedZWag2q

## Model and Resources

| Item | Value |
|------|-------|
| **Base model** | [`Qwen/Qwen2.5-7B-Instruct`](https://huggingface.co/Qwen/Qwen2.5-7B-Instruct) |
| **Workers** | 4 × L4 / A10G |

> Compared to SFT, DPO holds two copies of the model (policy + reference), and alignment datasets often use long contexts, so it's the ideal workflow for memory optimization techniques like **QLoRA**. On 24 GB NVIDIA L4 GPUs, running DPO at FP16 for 7B models generally OOMs without QLoRA.

### Understand the Dataset
For this tutorial, we will use [`ultrafeedback.jsonl`](https://huggingface.co/datasets/kaitchup/UltraFeedback-prompt-chosen-rejected), a JSONL preference dataset tailored for Direct Preference Optimization (DPO). Each sample contains one instruction **prompt** and two candidate completions: a **preferred** (`chosen`) response and a **less preferred** (`rejected`) response.

This dataset includes:
- `prompt`: An instruction/question to answer (often multi-sentence, with constraints).
- `chosen`: The response that best follows the instruction.
- `rejected`: A weaker alternative for the same prompt (may span multiple lines).

**Note:** Files are in JSON Lines format—one JSON object per line. Each record is independent (no multi-turn conversation thread).

<details>
  <summary>Dataset Example</summary>

  ```json
  {
    "prompt": "Paraphrase the given questions to have different wording. Your paraphrased questions should have the same answer as the original question. Try to change the sentence as much as possible using synonyms and/or rearranging the structure of the sentence. The questions are in three domains: presidents, national parks, and dogs. Each question has a keyword indicating its domain. Keywords are \"this national park\", \"this dog breed\", and \"this president\", which will be replaced with the name of an actual president, a national park, or a breed of dog. Hence, in paraphrasing, this keyword should also be used the same way. Do not write questions that compare or involve multiple domains. Do not write open-ended or subjective questions (e.g., questions that can be answered differently by different people.) Make your questions specific and concrete. Your question should have the same type of answer as the original question(e.g., if the question is extractive, the paraphrased question should be extractive as well.)\n\nWhat lakes are in this national park?",

    "rejected": "What bodies of water are located in this national park? \n\nWhich president is commonly known for his efforts to protect natural resources?\n\nWhich president is recognized for their dedication to preserving the environment? \n\nWhat type of dog breed is known for its loyalty and affectionate nature?\n\nWhat breed of dog is renowned for its faithfulness and loving personality?",
    
    "chosen": "Which bodies of water can be found within the borders of this particular national park?"
  }
  ```
</details>

### Register the local dataset

To specify new datasets that are accessible across Ray worker nodes, you must first add all dataset files and a `dataset_info.json` to **[storage shared across nodes](https://docs.anyscale.com/configuration/storage#shared)** such as `/mnt/cluster_storage`. 

For example, if you wanted to run DPO post-training on the `ultrafeedback` dataset locally, first go through the following setup steps:

`dataset_info.json`
```json
{
  "my_ultrafeedback": {
    "file_name": "ultrafeedback.jsonl",
    "ranking": true,
    "columns": {
      "prompt": "prompt",
      "chosen": "chosen",
      "rejected": "rejected"
    }
  }
}
```

For a more detailed dataset preparation and formatting guide, follow [_](https://docs.anyscale.com/llm/fine-tuning/data-preparation#data-format)

In [2]:
%%bash
# Make sure all files are accessible to worker nodes
# Create a copy of the data in /mnt/cluster_storage
wget https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/alpaca/ultrafeedback.jsonl -O /mnt/cluster_storage/ultrafeedback.jsonl
# Create a copy of the dataset registry in /mnt/cluster_storage
cp ../dataset-configs/dataset_info.json /mnt/cluster_storage/

--2025-09-19 15:54:37--  https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/datasets/alpaca/ultrafeedback.jsonl
Resolving anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)... 3.5.77.216, 52.218.230.41, 52.92.229.66, ...
Connecting to anyscale-public-materials.s3.us-west-2.amazonaws.com (anyscale-public-materials.s3.us-west-2.amazonaws.com)|3.5.77.216|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 291881 (285K) [application/x-www-form-urlencoded]
Saving to: ‘/mnt/cluster_storage/ultrafeedback.jsonl’

     0K .......... .......... .......... .......... .......... 17%  233M 0s
    50K .......... .......... .......... .......... .......... 35% 35.1M 0s
   100K .......... .......... .......... .......... .......... 52%  240M 0s
   150K .......... .......... .......... .......... .......... 70% 80.9M 0s
   200K .......... .......... .......... .......... .......... 87%  248M 0s
   250K .......... .......... .......... .....                100%  264M=0.003s

2025-09-19 15:54:37 (102 MB/s) - ‘/mnt/cluster_storage/ultrafeedback.jsonl’ saved [291881/291881]



## Step 3: Create the Preference-Tuning Config (DPO + QLoRA)

Next, create the YAML configuration file that defines your DPO (Direct Preference Optimization) run. It specifies the base model, quantization (QLoRA), dataset, DPO hyperparameters, logging, and Ray cluster resources.

Here is the `qwen2.5_7b_qlora_dpo_ray.yaml` included in the workspace:

**Important notes:**
- **QLoRA quantization:** `quantization_bit: 4` with `quantization_method: bnb` applies quantization using bitsandbytes, reducing memory while preserving quality. If you use a model *pre-quantized* with AWQ, **omit** these keys.
- **LoRA setup**: If you prefer standard LoRA, **disable quantization** by removing both `quantization_bit` and `quantization_method` from the config.
- **Access & paths:** The YAML only needs to be on the **head node**, but any referenced paths (`dataset_dir`, `output_dir`) must live on storage **reachable by all workers** (e.g., `/mnt/cluster_storage/`).
- **Gated models:** Qwen is generally ungated. For gated bases (e.g., Llama), add your `HF_TOKEN`.

### LLaMA-Factory + Ray Configuration

```yaml
# train-configs/dpo_qlora.yaml

### model
trust_remote_code: true
model_name_or_path: Qwen/Qwen2.5-7B-Instruct

### method
# If you instead want to use just LoRA, or use a pre-quantized model like Qwen/Qwen2.5-7B-Instruct-AWQ, then omit the quantization_bit/method keys below
quantization_bit: 4 # 4-bit base weights (QLoRA). Use 8 for 8-bit; omit for FP16/BF16
quantization_method: bnb  # QLoRA via BitsAndBytes or hqq / eetq

stage: dpo
do_train: true
finetuning_type: lora
lora_rank: 8
lora_target: all
pref_beta: 0.1
pref_loss: sigmoid  # choices: [sigmoid (dpo), orpo, simpo]

# local dataset
dataset: my_ultrafeedback
dataset_dir: /mnt/cluster_storage

template: qwen
cutoff_len: 1024
max_samples: 1000
overwrite_cache: true
preprocessing_num_workers: 16

### output
output_dir: qwen2.5_7b_qlora_dpo
logging_steps: 5
save_steps: 5              # for tensorboard logging purpose too, can increase if not using tensorboard
plot_loss: true
report_to: tensorboard  # or none

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 2
num_train_epochs: 3.0  # low for demo purpose; adjust as needed
learning_rate: 5.0e-6
bf16: true
lr_scheduler_type: cosine
warmup_ratio: 0.1
ddp_timeout: 180000000

### ray
ray_run_name: qwen2.5_7b_qlora_dpo
ray_storage_path: /mnt/cluster_storage/
ray_num_workers: 4  # Number of GPUs to use.
resources_per_worker:
  GPU: 1
  anyscale/accelerator_shape:4xL4: 0.001  # Use this to specify a specific node shape.
  # accelerator_type:L4: 0.001            # Or use this to simply specify a GPU type.
  # See https://docs.ray.io/en/master/ray-core/accelerator-types.html#accelerator-types for a full list of accelerator types.

ray_init_kwargs:
  runtime_env:
    env_vars:
      # if using gated models like meta-llama/Llama-3.1-8B-Instruct
      # HF_TOKEN: <your_huggingface_token>
      # Enable faster downloads if hf_transfer is installed:
      HF_HUB_ENABLE_HF_TRANSFER: '1'
```

## Step 4: Train and Monitor

With all configuration in place, you can launch fine-tuning/post-training in one of two ways.

### Option A — Run from a Workspace (quick start)

The `USE_RAY=1` prefix tells LLaMA-Factory to run in distributed mode on the Ray cluster attached to your workspace.

In [ ]:
%%bash
USE_RAY=1 llamafactory-cli train ../train-configs/dpo_qlora.yaml

INFO 09-19 16:12:06 [__init__.py:248] No platform detected, vLLM is running on UnspecifiedPlatform
WARNING 09-19 16:12:06 [_custom_ops.py:20] Failed to import from vllm._C with ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


2025-09-19 16:12:10,738	INFO worker.py:1747 -- Connecting to existing Ray cluster at address: 10.0.168.141:6379...
2025-09-19 16:12:10,748	INFO worker.py:1918 -- Connected to Ray cluster. View the dashboard at https://session-c3rc1dvuypysehcmb91gu17t54.i.anyscaleuserdata.com 
2025-09-19 16:12:10,750	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_4743ba917c5b0b6789a61fbd792b5972f2c8ed63.zip' (0.10MiB) to Ray cluster...
2025-09-19 16:12:10,751	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_4743ba917c5b0b6789a61fbd792b5972f2c8ed63.zip'.



View detailed results here: /mnt/cluster_storage/qwen2.5_7b_qlora_dpo
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-09-19_15-49-42_124608_2430/artifacts/2025-09-19_16-12-10/qwen2.5_7b_qlora_dpo/driver_artifacts`

Training started with configuration:
╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Training config                                                                                              │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ train_loop_config/args/bf16                                                                             True │
│ train_loop_config/args/cutoff_len                                                                       1024 │
│ train_loop_config/args/dataset                                                              my_ultrafeedback │
│ train_loop_config/args

(RayTrainWorker pid=6857, ip=10.0.183.178) Setting up process group for: env:// [rank=0, world_size=4]
(TorchTrainer pid=6752, ip=10.0.183.178) Started distributed worker processes: 
(TorchTrainer pid=6752, ip=10.0.183.178) - (node_id=c87e4ad0b35e4f468ebff6d2970822063a67672f55ac6702a530230f, ip=10.0.183.178, pid=6857) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=6752, ip=10.0.183.178) - (node_id=c87e4ad0b35e4f468ebff6d2970822063a67672f55ac6702a530230f, ip=10.0.183.178, pid=6858) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=6752, ip=10.0.183.178) - (node_id=c87e4ad0b35e4f468ebff6d2970822063a67672f55ac6702a530230f, ip=10.0.183.178, pid=6859) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=6752, ip=10.0.183.178) - (node_id=c87e4ad0b35e4f468ebff6d2970822063a67672f55ac6702a530230f, ip=10.0.183.178, pid=6856) world_rank=3, local_rank=3, node_rank=0


(RayTrainWorker pid=6857, ip=10.0.183.178) [WARNING|2025-09-19 16:12:24] llamafactory.hparams.parser:148 >> We recommend enable `upcast_layernorm` in quantized training.
(RayTrainWorker pid=6857, ip=10.0.183.178) [INFO|2025-09-19 16:12:24] llamafactory.hparams.parser:143 >> Set `ddp_find_unused_parameters` to False in DDP training since LoRA is enabled.
(RayTrainWorker pid=6857, ip=10.0.183.178) [INFO|2025-09-19 16:12:24] llamafactory.hparams.parser:406 >> Process rank: 0, world size: 4, device: cuda:0, distributed training: True, compute dtype: torch.bfloat16


...

(RayTrainWorker pid=6857, ip=10.0.183.178) ***** train metrics *****
(RayTrainWorker pid=6857, ip=10.0.183.178)   epoch                    =        3.0
(RayTrainWorker pid=6857, ip=10.0.183.178)   total_flos               = 12512612GF
(RayTrainWorker pid=6857, ip=10.0.183.178)   train_loss               =      0.661
(RayTrainWorker pid=6857, ip=10.0.183.178)   train_runtime            = 0:03:51.38
(RayTrainWorker pid=6857, ip=10.0.183.178)   train_samples_per_second =      1.297
(RayTrainWorker pid=6857, ip=10.0.183.178)   train_steps_per_second   =      0.169
(RayTrainWorker pid=6857, ip=10.0.183.178) Figure saved at: qwen2.5_7b_qlora_dpo/training_loss.png
(RayTrainWorker pid=6857, ip=10.0.183.178) Figure saved at: qwen2.5_7b_qlora_dpo/training_rewards_accuracies.png
(RayTrainWorker pid=6857, ip=10.0.183.178) [WARNING|2025-09-19 16:16:35] llamafactory.extras.ploting:148 >> No metric eval_loss to plot.


(RayTrainWorker pid=6857, ip=10.0.183.178) [INFO|modelcard.py:450] 2025-09-19 16:16:35,095 >> Dropping the following result as it does not have all the necessary fields:
(RayTrainWorker pid=6857, ip=10.0.183.178) {'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}}



Training completed after 8 iterations at 2025-09-19 16:16:36. Total running time: 4min 25s


2025-09-19 16:16:36,549	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/mnt/cluster_storage/qwen2.5_7b_qlora_dpo' in 0.0220s.


(RayTrainWorker pid=6857, ip=10.0.183.178) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/mnt/cluster_storage/qwen2.5_7b_qlora_dpo/TorchTrainer_12134_00000_0_2025-09-19_16-12-10/checkpoint_000007) [repeated 3x across cluster]


### Option B — Run as an Anyscale Job (production)

For longer or production runs, submit the training as an **Anyscale Job**. Jobs run outside your interactive session for better stability, retries, and durable logs. You’ll package LLaMA-Factory and other libraries in a container image and launch with a short job config. See [_](https://docs.anyscale.com/llm/fine-tuning/llamafactory-jobs) for the step-by-step guide.

### Monitoring with TensorBoard
If you enabled TensorBoard logging (`report_to: tensorboard` in your YAML), you can watch metrics (e.g., training loss) update live and compare multiple runs with the same run name side-by-side.

- **While the job is running:** LLaMA-Factory prints a ready-to-run command that starts with `tensorboard --logdir`. Open a new terminal and run it. Example:
  ```bash
  tensorboard --logdir /tmp/ray/session_*/artifacts/*/qwen2.5_7b_qlora_dpo/driver_artifacts
  ```

- **After the job (shared storage):** Point TensorBoard at `{ray_storage_path}/{ray_run_name}/`. Each `TorchTrainer_*` subfolder holds event files for a single run. Using the parent folder aggregates all runs for easy comparison.
  ```bash
  tensorboard --logdir /mnt/cluster_storage/qwen2.5_7b_qlora_dpo
  ```

In your Anyscale workspace, look for the open **port 6006** labeled **TensorBoard** to view the dashboards.
![Anyscale workspace showing open ports with TensorBoard on port 6006](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/open-ports.png)

**TensorBoard**
![TensorBoard](https://anyscale-public-materials.s3.us-west-2.amazonaws.com/llm-finetuning/llama-factory/3.2.2/3.2.2-tensorboard.png)

For a more detailed guide on tracking experiments with other tools such as WandB or MLFlow, see [_](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking.md).

## Step 5: Locate Checkpoints

Checkpoints are written under `ray_storage_path/ray_run_name`. In this example run, the path is: `/mnt/cluster_storage/qwen2.5_32b_lora_sft`. 

Inside, you’ll see a **trainer session** directory named like:
`TorchTrainer_ff224_00000_0_2025-09-19_15-57-20/`.

- `TorchTrainer_*` is created **when the trainer starts**; the suffix encodes a short run id and the **start timestamp**.
- Within that directory, checkpoints are named `checkpoint_000xxx/`, where the number is the saved ordered checkpoints. 

The save cadence is controlled by `save_strategy` and `save_steps`. For instructions on how to resume interrupted training via `resume_from_checkpoint` and more, see [_](https://docs.anyscale.com/llm/fine-tuning/checkpointing#artifacts-directory).

## Step 6: Export the Model

If you use LoRA, you can keep the base model and adapter separate ([_](https://docs.anyscale.com/llm/serving/multi-lora)) or merge the adapter into the base model for low-latency inference. 

For full fine-tuning or freeze-tuning, export the fine-tuned model directly.

You may optionally apply post-training quantization on merged or full models before serving. See [_](https://docs.anyscale.com/llm/fine-tuning/observability-and-tracking.md#ptq) for the exact export commands and options.